In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn geopy regex

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re

In [ ]:
# Mostrar todas las filas pandas
pd.set_option('display.max_rows', None)
# Mostrar todas las columnas pandas
pd.set_option('display.max_columns', None)

In [ ]:
# Cargamos nuestros datasets
properties = pd.read_csv('data_propiedades.csv', encoding='utf-8', header=0)
marginality = pd.read_csv('result_com_col_cp_dropdup.csv', encoding='utf-8', header=0)
len(properties)

1416

In [ ]:
## Comenzamos el preprocesamiento para poder unir los datos de los dos dataframes
## Inicialmente obtendremos el codigo postal desde la ubicacion para cada propiedad

# Creamos una funcion que reciba la ubicacion y retorna el codigo postal usnado regex
def get_cp(location):
    pattern = r'[0-9]{5}'
    return re.findall(pattern, location)[0]

# Creamos la nueva columna cp aplicando la funcion a la columna ubicacion
properties['cp'] = properties['ubicacion'].apply(get_cp)
print(len(properties))
properties.head()

1416


,terreno,construccion,recamaras,banos,estacionamientos,ubicacion,precio,anio_construct,tipo,cp
0,61.5,61.5,2.0,1.0,1,"Doctores, Cuauhtémoc, Ciudad De México 06720, ...","$2,900,000 MXN",2022.0,regular,06720
1,55.2,55.2,1.0,1.0,1,"Doctores, Cuauhtémoc, Ciudad De México 06720, ...","$3,200,000 MXN",2018.0,regular,06720
2,60.0,60.0,3.0,1.0,0,"Culhuacán CTM Sección VIII, Coyoacán, Ciudad D...","$1,500,000 MXN",1998.0,departamento,04831
3,42.0,42.0,1.0,1.0,0,"Roma Norte, Cuauhtémoc, Ciudad De México 06700...","$2,137,000 MXN",1972.0,Duela,06700
4,121.0,220.0,3.0,2.0,1,"Condesa, Cuauhtémoc, Ciudad De México 06140, M...","$14,890,000 MXN",1964.0,regular,06140


In [ ]:
## Ahora procedemos a formatear los codigos postales de el dataset de marginalidad
## que se guardaron como enteros por lo cual algunos perdieron 0 a la izquierda
def format_cp(cp):
    return str(cp).zfill(5)
marginality['cp'] = marginality['CP'].apply(format_cp)
display(marginality.head())

## Tambien hay casos donde el codigo postal se repite en varias colonias por que
## tiene varias secciones, para este caso agruparemos por codigo postal
## y nos quedaremos con los primeros campos encontrados y para el caso de el campo
## población realizaremos una sumatoria para la poblacion total del codigo postal
marginality_grouped = marginality.groupby(['cp']).agg({'alcaldia':'first','colonia':'first',
                                                       'grado_marginalidad':'max',
                                                       'pob_2010':'sum',
                                                       'tipo_seccion':'first',
                                                       'tipo_col':'first'
                                                       }).reset_index()
marginality_grouped.head()


,alcaldia,cve_col,colonia,pob_2010,grado_marginalidad,CP,tipo_seccion,tipo_col,cp
0,AZCAPOTZALCO,02-001,AGUILERA,2014.0,4,2900,URBANO(A),COLONIA,02900
1,AZCAPOTZALCO,02-002,ALDANA,3378.0,5,2910,URBANO(A),COLONIA,02910
2,AZCAPOTZALCO,02-005,ANGEL ZIMBRON,2737.0,2,2099,URBANO(A),COLONIA,02099
3,AZCAPOTZALCO,02-006,ARENAL,4817.0,4,2980,URBANO(A),COLONIA,02980
4,AZCAPOTZALCO,02-007,CENTRO DE AZCAPOTZALCO,3043.0,1,2000,URBANO(A),COLONIA,02000


,cp,alcaldia,colonia,grado_marginalidad,pob_2010,tipo_seccion,tipo_col
0,01000,ALVARO OBREGON,SAN ANGEL INN,1,9630.0,URBANO(A),COLONIA
1,01010,ALVARO OBREGON,ALPES,2,5882.0,URBANO(A),COLONIA
2,01020,ALVARO OBREGON,GUADALUPE INN,1,5743.0,URBANO(A),COLONIA
3,01030,ALVARO OBREGON,AXOTLA,1,7894.0,URBANO(A),COLONIA
4,01040,ALVARO OBREGON,TLACOPAC,1,3025.0,URBANO(A),COLONIA


In [ ]:
## Con esto ya podemos realizar un cruce a partir de nuestros datos
df_complete = properties.merge(marginality_grouped, on='cp', how='left')
print(len(df_complete))
# y como paso final borramos todos aquellos que cuenten con datos nan
# ya sea por que no se contaba con ellos en la extraccion o porque no se cuenta
# con datos de marginalidad al hacer merge
df_complete = df_complete.dropna()
df_complete = df_complete[df_complete['precio'] != 'A consultar']
# Podemos ver que perdimos un numero considerable de registros
print(len(df_complete))

1416
1068


In [ ]:
# Aunque perdimos un numero considerable de propiedades aun podemos trabajar con el numero restante
# en este caso 1000 propiedades solo para cdmx nos pueden aportar buena informacioón aun
# comenzamos dandole un vistazo a la info del dataframe
df_complete.info(),df_complete.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 1068 entries, 0 to 1415
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   terreno             1068 non-null   object 
 1   construccion        1068 non-null   object 
 2   recamaras           1068 non-null   float64
 3   banos               1068 non-null   float64
 4   estacionamientos    1068 non-null   int64  
 5   ubicacion           1068 non-null   object 
 6   precio              1068 non-null   object 
 7   anio_construct      1068 non-null   float64
 8   tipo                1068 non-null   object 
 9   cp                  1068 non-null   object 
 10  alcaldia            1068 non-null   object 
 11  colonia             1068 non-null   object 
 12  grado_marginalidad  1068 non-null   float64
 13  pob_2010            1068 non-null   float64
 14  tipo_seccion        1068 non-null   object 
 15  tipo_col            1068 non-null   object 
dtypes: float64(

(None,
          recamaras        banos  estacionamientos  anio_construct  \
 count  1068.000000  1068.000000       1068.000000     1068.000000   
 mean      3.013109     2.422285          2.331461     1587.982210   
 std       1.502830     1.373381          6.900733      804.381813   
 min       1.000000     1.000000          0.000000        1.000000   
 25%       2.000000     2.000000          1.000000     1969.000000   
 50%       3.000000     2.000000          2.000000     1995.000000   
 75%       3.000000     3.000000          2.000000     2016.000000   
 max      15.000000    17.000000        215.000000     2025.000000   
 
        grado_marginalidad       pob_2010  
 count         1068.000000    1068.000000  
 mean             2.538390   14059.282772  
 std              1.351672   14576.235171  
 min              1.000000     355.000000  
 25%              1.000000    5248.750000  
 50%              2.000000    9815.000000  
 75%              4.000000   17464.000000  
 max     

In [ ]:
## Damos el formato correspondiente a algunas columnas
df_complete['terreno']=df_complete['terreno'].apply(lambda x: x.replace(',','')).astype(float)
df_complete['construccion']=df_complete['construccion'].apply(lambda x: x.replace(',','')).astype(float)
# Pasamos el codigo postal a int, ya que si trataramos despues de hacer un one hot tendriamos demasiados elementos
df_complete['cp'] = df_complete['cp'].astype(int)
# Revisando el contenido de la columna tipo no nos aporta mucho ya que cada asesor
# carga a su entendimiento este campo y no define mucho
df_complete.drop('tipo', axis=1, inplace=True)
# Para el anio encontramos que hay años en los cuales solo cuentan con dos digitos
# ejemplo 01 o 24, para este caso se refieren al 2001 y 2024
def fix_year(year):
    if year <= 24:
        return 2000 + year
    elif year > 24 and year < 100:
        return 1900 + year
    elif year >= 200 and year <= 224:
        return 2000 + year%100
    else:
        return year
df_complete['anio_construct'] = df_complete['anio_construct'].apply(fix_year)
## Para el caso del precio viene como cadena por lo cual lo pasaremos a flotante
def extract_price(price):
    price = price.replace(',','').strip()
    price = re.findall(r'[0-9]+', price)[0]
    return float(price)
# Guardamos un respaldo por alguna mala conversion
df_complete['precio_str'] = df_complete['precio']
# Extraemos los precios de tipo flotante
df_complete['precio'] = df_complete['precio'].apply(extract_price)


In [ ]:
# le damos de nuevo un vistazo a la info del dataframe
df_complete.info(),df_complete.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 1068 entries, 0 to 1415
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   terreno             1068 non-null   float64
 1   construccion        1068 non-null   float64
 2   recamaras           1068 non-null   float64
 3   banos               1068 non-null   float64
 4   estacionamientos    1068 non-null   int64  
 5   ubicacion           1068 non-null   object 
 6   precio              1068 non-null   float64
 7   anio_construct      1068 non-null   float64
 8   cp                  1068 non-null   int64  
 9   alcaldia            1068 non-null   object 
 10  colonia             1068 non-null   object 
 11  grado_marginalidad  1068 non-null   float64
 12  pob_2010            1068 non-null   float64
 13  tipo_seccion        1068 non-null   object 
 14  tipo_col            1068 non-null   object 
 15  precio_str          1068 non-null   object 
dtypes: float64(

(None,
            terreno  construccion    recamaras        banos  estacionamientos  \
 count  1068.000000   1068.000000  1068.000000  1068.000000       1068.000000   
 mean    224.533521    206.178933     3.013109     2.422285          2.331461   
 std     392.572682    187.582327     1.502830     1.373381          6.900733   
 min      18.000000     30.000000     1.000000     1.000000          0.000000   
 25%      75.000000     79.000000     2.000000     2.000000          1.000000   
 50%     125.000000    140.000000     3.000000     2.000000          2.000000   
 75%     218.000000    270.000000     3.000000     3.000000          2.000000   
 max    7671.000000   2000.000000    15.000000    17.000000        215.000000   
 
              precio  anio_construct            cp  grado_marginalidad  \
 count  1.068000e+03     1068.000000   1068.000000         1068.000000   
 mean   8.398544e+06     1996.877341   6680.009363            2.538390   
 std    1.056286e+07       25.758465   4

In [ ]:
df_complete.reset_index(drop=True, inplace=True)
df_complete.head()

,terreno,construccion,recamaras,banos,estacionamientos,ubicacion,precio,anio_construct,cp,alcaldia,colonia,grado_marginalidad,pob_2010,tipo_seccion,tipo_col,precio_str
0,61.5,61.5,2.0,1.0,1,"Doctores, Cuauhtémoc, Ciudad De México 06720, ...",2900000.0,2022.0,6720,CUAUHTEMOC,DOCTORES I,5.0,44703.0,URBANO(A),COLONIA,"$2,900,000 MXN"
1,55.2,55.2,1.0,1.0,1,"Doctores, Cuauhtémoc, Ciudad De México 06720, ...",3200000.0,2018.0,6720,CUAUHTEMOC,DOCTORES I,5.0,44703.0,URBANO(A),COLONIA,"$3,200,000 MXN"
2,42.0,42.0,1.0,1.0,0,"Roma Norte, Cuauhtémoc, Ciudad De México 06700...",2137000.0,1972.0,6700,CUAUHTEMOC,ROMA NORTE I,3.0,27770.0,URBANO(A),COLONIA,"$2,137,000 MXN"
3,121.0,220.0,3.0,2.0,1,"Condesa, Cuauhtémoc, Ciudad De México 06140, M...",14890000.0,1964.0,6140,CUAUHTEMOC,CONDESA,2.0,11657.0,URBANO(A),COLONIA,"$14,890,000 MXN"
4,122.0,172.0,6.0,2.0,0,"Reynosa Tamaulipas, Azcapotzalco, Ciudad De Mé...",3300000.0,1980.0,2200,AZCAPOTZALCO,REYNOSA TAMAULIPAS,5.0,7490.0,URBANO(A),COLONIA,"$3,300,000 MXN"


In [ ]:
# Para fines exploratorios trataremos de conseguir la latitud y longitud aproximada de
# cada inmueble

from geopy.geocoders import Nominatim

# Crear una instancia del geocodificador
geolocator = Nominatim(user_agent="proyectoFinalInmueblesCDMX")
counter_errors = 0
# Obtener la ubicación
def get_lat_long(cp):
    try:
        location = geolocator.geocode(f'Ciudad de Mexico {str(cp).zfill(5)}')
        return (location.latitude,location.longitude)
    except:
        global counter_errors
        counter_errors = counter_errors + 1
        return (np.nan,np.nan)
df_complete['lat_long'] = df_complete['cp'].apply(get_lat_long)
df_complete.head()

,terreno,construccion,recamaras,banos,estacionamientos,ubicacion,precio,anio_construct,cp,alcaldia,colonia,grado_marginalidad,pob_2010,tipo_seccion,tipo_col,precio_str,lat_long
0,61.5,61.5,2.0,1.0,1,"Doctores, Cuauhtémoc, Ciudad De México 06720, ...",2900000.0,2022.0,6720,CUAUHTEMOC,DOCTORES I,5.0,44703.0,URBANO(A),COLONIA,"$2,900,000 MXN","(19.4326296, -99.1331785)"
1,55.2,55.2,1.0,1.0,1,"Doctores, Cuauhtémoc, Ciudad De México 06720, ...",3200000.0,2018.0,6720,CUAUHTEMOC,DOCTORES I,5.0,44703.0,URBANO(A),COLONIA,"$3,200,000 MXN","(19.4326296, -99.1331785)"
2,42.0,42.0,1.0,1.0,0,"Roma Norte, Cuauhtémoc, Ciudad De México 06700...",2137000.0,1972.0,6700,CUAUHTEMOC,ROMA NORTE I,3.0,27770.0,URBANO(A),COLONIA,"$2,137,000 MXN","(19.4326296, -99.1331785)"
3,121.0,220.0,3.0,2.0,1,"Condesa, Cuauhtémoc, Ciudad De México 06140, M...",14890000.0,1964.0,6140,CUAUHTEMOC,CONDESA,2.0,11657.0,URBANO(A),COLONIA,"$14,890,000 MXN","(19.4326296, -99.1331785)"
4,122.0,172.0,6.0,2.0,0,"Reynosa Tamaulipas, Azcapotzalco, Ciudad De Mé...",3300000.0,1980.0,2200,AZCAPOTZALCO,REYNOSA TAMAULIPAS,5.0,7490.0,URBANO(A),COLONIA,"$3,300,000 MXN","(19.4326296, -99.1331785)"


In [ ]:
## Vemos cuantas latitudes y logitudes son diferente a na
df_complete[df_complete['lat_long']==(np.nan,np.nan)].head()
counter_errors

0

In [ ]:
get_lat_long('Ciudad De México 04800')

(19.4326296, -99.1331785)

In [ ]:
# Guardamos nuestro dataframe hasta este punto para no estar retrabajando
# en este punto ya contamos con datos que podemos comenzar a analizar
df_complete.to_csv('df_complete.csv', index=False)

In [ ]:
# Separamos latitud y longitud en columnas separas y guardamos tambien este df
df_complete['lat'] = df_complete['lat_long'].apply(lambda x: x[0])
df_complete['long'] = df_complete['lat_long'].apply(lambda x: x[1])
df_complete.to_csv('df_complete_lat_long.csv', index=False)